In [1]:
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup


# Web Scraping

In [2]:
pd.set_option('display.max_rows', 500)

In [3]:
data_path= "../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
pd.read_csv(data_path)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,38113,38129,38140,38143,38162,38165,38196,38243,38288,38304
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,8927,9083,9195,9279,9380,9513,9606,9728,9844,9967
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,42619,43016,43403,43781,44146,44494,44833,45158,45469,45773
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,1098,1098,1124,1124,1124,1176,1184,1199,1199,1215
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,2332,2415,2471,2551,2624,2654,2729,2777,2805,2876
5,NaN,Antigua and Barbuda,17.060800,-61.796400,0,0,0,0,0,0,...,94,94,94,94,94,94,94,94,95,95
6,NaN,Argentina,-38.416100,-63.616700,0,0,0,0,0,0,...,370188,380292,392009,401239,408426,417735,428239,439172,451198,461882
7,NaN,Armenia,40.069100,45.038200,0,0,0,0,0,0,...,43067,43270,43451,43626,43750,43781,43878,44075,44271,44461
8,Australian Capital Territory,Australia,-35.473500,149.012400,0,0,0,0,0,0,...,113,113,113,113,113,113,113,113,113,113
9,New South Wales,Australia,-33.868800,151.209300,0,0,0,0,3,4,...,4006,4019,4033,4040,4050,4063,4079,4091,4099,4104


In [4]:
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")
soup = BeautifulSoup(page.content, 'html.parser')
soup.find('table')
html_table=soup.find('table') 
all_rows=html_table.find_all('tr')


In [5]:
final_data_list=[]
for pos,rows in enumerate(all_rows):
   
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] #td for data element
    final_data_list.append(col_list)

In [6]:
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                       1:'cases',
                                                       2:'changes',
                                                       3:'cases_per_100k',
                                                       4:'fatal',
                                                       5:'comment'})

In [7]:
pd_daily_status

,state,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,43.423,+164,1.438,"13,0",1.865
3,Bayern,59.014,+322,1719,"13,1",2.642
4,Berlin,11.703,+80,493,"13,2",226
5,Branden­burg,3.927,+4,49,"2,0",169
6,Bremen,2.067,+11,78,"11,4",58
7,Hamburg,6.486,+35,204,"11,1",267
8,Hessen,16.098,+120,658,"10,5",535
9,Meck­lenburg-Vor­pommern,1.023,0,16,"1,0",20
10,Nieder­sachsen,17.236,+26,461,"5,8",665
11,Nord­rhein-West­falen,60.257,+172,1.569,"8,7",1.823


# API Calls

In [8]:
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [9]:
json_object=json.loads(data.content)
json_object.keys()


dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'serverGens', 'geometryType', 'spatialReference', 'fields', 'features'])

In [10]:
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])
    

In [11]:
pd_full_list=pd.DataFrame(full_list)
pd_full_list

,AGS_TXT,Aktualisierung,Death,Fallzahl,GlobalID,LAN_ew_AGS,LAN_ew_BEZ,LAN_ew_EWZ,LAN_ew_GEN,OBJECTID,OBJECTID_1,Shape__Area,Shape__Length,faelle_100000_EW
0,01,1599343200000,161,4120,fc5ba936-c95c-432c-8a33-9eb2f30b660f,01,Land,2896712,Schleswig-Holstein,15,1,4.573731e+10,2.881496e+06,142.230225
1,02,1599343200000,267,6486,0f3e860c-5181-4d3f-a421-1d51f50315ea,02,Freie und Hansestadt,1841179,Hamburg,6,2,2.089396e+09,4.188002e+05,352.274276
2,03,1599343200000,665,17236,3fd77024-c29b-4843-9be8-682ad48e60c9,03,Land,7982448,Niedersachsen,9,3,1.299836e+11,4.008988e+06,215.923737
3,04,1599343200000,58,2067,4132268b-54de-4327-ac1e-760e915112f1,04,Freie Hansestadt,682986,Bremen,5,4,1.119157e+09,3.357177e+05,302.641635
4,05,1599343200000,1823,60257,561d658f-3ee5-46e3-bc95-3528c6558ab9,05,Land,17932651,Nordrhein-Westfalen,10,5,8.782936e+10,2.648673e+06,336.018361
5,06,1599343200000,535,16098,93277ac4-e8fc-48c7-8940-028dc2ed66af,06,Land,6265809,Hessen,7,6,5.235913e+10,2.148244e+06,256.918141
6,07,1599343200000,243,9324,e9b4296f-9be2-4e53-9a58-ccf1396cb03d,07,Land,4084844,Rheinland-Pfalz,11,7,4.783877e+10,1.774430e+06,228.258411
7,08,1599343200000,1865,43423,80394ddf-c6a4-4a6e-be8e-0259a81b22a9,08,Land,11069533,Baden-Württemberg,1,8,8.151732e+10,2.544320e+06,392.274904
8,09,1599343200000,2642,59014,1ff920f4-62cd-4a4f-b8c9-f042f2a3e00a,09,Freistaat,13076721,Bayern,2,9,1.634855e+11,3.898618e+06,451.290503
9,10,1599343200000,174,3197,e3396a6f-8a30-4fdf-8df7-def77dd38bea,10,Land,990509,Saarland,12,10,6.060692e+09,5.626789e+05,322.763347


In [12]:
headers = {
    'Cache-Control': 'no-cache',
    'Subscription-Key': '28ee4219700f48718be78b057beb7eb4',
}

response = requests.get('https://api.smartable.ai/coronavirus/stats/US', headers=headers)
print(response)

<Response [200]>


In [13]:
US_dict=json.loads(response.content) 
with open('../data/raw/SMARTABLE/US_data.json', 'w') as outfile:
    json.dump(US_dict, outfile,indent=2)

In [14]:
print(json.dumps(US_dict,indent=2))

{
  "location": {
    "long": -95.712891,
    "countryOrRegion": "United States",
    "provinceOrState": null,
    "county": null,
    "isoCode": "US",
    "lat": 37.09024
  },
  "updatedDateTime": "2020-09-06T14:24:39.9536698Z",
  "stats": {
    "totalConfirmedCases": 6368030,
    "newlyConfirmedCases": 7497,
    "totalDeaths": 200027,
    "newDeaths": 90,
    "totalRecoveredCases": 2796278,
    "newlyRecoveredCases": 0,
    "history": [
      {
        "date": "2020-01-22T00:00:00",
        "confirmed": 1,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-23T00:00:00",
        "confirmed": 1,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-24T00:00:00",
        "confirmed": 2,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-25T00:00:00",
        "confirmed": 2,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-26T00:00:00",
        "confi